In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

"""from sklearn.model_selection import train_test_split
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer"""

from sklearn.preprocessing import OneHotEncoder


In [2]:
os.chdir("../")
from src.data import import_data

In [3]:
import_data()

,Customer_ID,upsell_xsell,churn,lifetime_value,avg_arpu_3m,acct_age,billing_cycle,nbr_contracts_ltd,credit_class,sales_channel,...,pymts_late_ltd,calls_care_ltd,MB_Data_Usg_M04,MB_Data_Usg_M05,MB_Data_Usg_M06,MB_Data_Usg_M07,MB_Data_Usg_M08,MB_Data_Usg_M09,seconds_of_data_norm,seconds_of_data_log
0,471.0,0.0,0.0,1693.8,44.980000,55.000000,4.0,7.0,b'prime',b'Indirect',...,5.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,4.110874
1,1057.0,0.0,0.0,38682.5,44.989330,47.000000,3.0,5.0,b'prime',b'Indirect',...,5.0,115.0,0.0,0.0,0.0,0.0,330.0,646.0,48300.0,10.785208
2,1177.0,0.0,0.0,5526.4,51.775434,46.176471,7.0,4.8,b'prime',b'Retail',...,5.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,14700.0,9.595671
3,1181.0,0.0,0.0,505.6,51.775434,75.000000,7.0,4.8,b'prime',b'Retail',...,5.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,1185.0,0.0,1.0,9143.2,83.320000,50.000000,5.0,3.0,b'prime',b'Indirect',...,5.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,15840.0,9.670357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56552,3999397.0,0.0,0.0,1151.3,54.980000,58.000000,11.0,4.0,b'prime',b'Indirect',...,0.0,67.0,1562.0,51.0,516.0,1.0,70.0,0.0,6142.0,8.723069
56553,3999465.0,0.0,0.0,3672.8,37.769990,69.000000,11.0,2.0,b'prime',b'National Sales',...,4.0,68.0,137.0,4.0,342.0,331.0,592.0,46.0,7625.0,8.939319
56554,3999574.0,0.0,0.0,1112.8,54.655000,52.000000,7.0,3.0,b'risky',b'National Sales',...,0.0,71.0,188.0,163.0,4.0,11.0,1.0,16.0,1113.0,7.015712
56555,3999770.0,0.0,0.0,1126.5,64.980000,29.000000,1.0,2.0,b'prime',b'Indirect',...,8.0,77.0,0.0,2.0,1.0,0.0,3.0,1.0,3089.0,8.035926


# Idea Testing

In [ ]:
comms_df = pd.read_sas("./inputs/raw/commsdata.sas7bdat", encoding="utf-8")

In [ ]:
comms_df.avg_arpu_3m.mode()

In [ ]:
MetadataStats(comms_df).variable_summarizing("call_category_1")

In [ ]:
MetadataStats(comms_df).metadata_report('char')


In [ ]:
comms_df["avg_arpu_3m"].skew()

In [ ]:
df_ = comms_df["verbatims"].to_frame()
df_["verbatims_process"] = df_["verbatims"].apply(lambda x: x.decode("utf-8"))

In [ ]:
raise SystemExit("Stop exec!")

In [ ]:
comms_df.shape

In [ ]:
comms_df

In [ ]:
pd.qcut(comms_df["calls_care_ltd"], 4, duplicates='drop').astype(str)

In [ ]:
comms_df.isna().sum().mean()

In [ ]:
l_object_vars = comms_df.select_dtypes(include='object').columns.to_list()
l_object_vars.remove('verbatims')
l_object_vars.remove('resolution')
df_obj = comms_df[["credit_class", "sales_channel"]].apply(lambda x: x.apply(lambda z: z.decode("utf-8")), axis=1)
df_obj

In [ ]:
docs = df_["verbatims_process"].to_list()

In [ ]:
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
topic_model = BERTopic(ctfidf_model=ctfidf_model)
topics, probs = topic_model.fit_transform(docs)
topic_model.reduce_topics(docs, nr_topics=15)
vectorizer_model = CountVectorizer(stop_words="english")
topic_model.update_topics(docs, n_gram_range=(1, 3), vectorizer_model=vectorizer_model)

In [ ]:
topic_model.get_document_info(docs)

In [ ]:
topic_model.get_topics()[-1]

In [ ]:
"""data = import_from_local(".")
MetaDataManagement(data).metadata_management_pipeline()
st.dataframe(data.head())

st.markdown("# Categorical features processing")


### splitting
X_train, X_valid, y_train, y_valid = train_valid_split(data)
    
## data variable management pipeline
DataManagement(X_train).data_management_pipeline()
DataManagement(X_valid).data_management_pipeline()


# models pipelines
## random forest model
evaluate_rdmf(X_train, X_valid, y_train, y_valid, n_estimators=20)

# model comparaison"""